# Projeto Don - Desafio Felicidade 2021
Uma visão geral mundial e uma análise um pouco mais profunda sobre o Brasil no Relatório de Felicidade Mundial de 2021.  
FAQ: https://worldhappiness.report/faq/  

Projetos que nos inspiraram:
* https://www.kaggle.com/joshuaswords/awesome-eda-2021-happiness-population/ 
* https://www.kaggle.com/ajaypalsinghlo/world-happiness-report-2021

## Introdução

A principal intenção deste projeto é aprender mais sobre a área de Ciência de dados e testar nosso conhecimento através de publicações e troca de conhecimento. O tema desse projeto é como está o mundo em relação a felicidade e, em específico, como o Brasil está neste cenário em relação aos outros. O _notebook_ baseia-se no Relatório de Felicidade Mundial de 2021 e dos anos anteriores.

# Setup dos Dados
Vamos transformar os dados em DataFrames do Pandas. Além disso, vamos renomear algumas colunas para ficar mais fácil e nos DataFrames df_1 e df_2, vamos usar os nomes dos países como os índices do DataFrame.

In [ ]:
# Importando as bibliotecas que vão nos ajudar a processar e visualizar os dados.
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")        
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
import seaborn as sns
import squarify

# Importando os dados
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# Pegando os dados de 2021 e o histórico
df = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv')
df2 = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report.csv')
# Fazendo a mesma coisa, mas utilizando os países como índices.
df_1 = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report-2021.csv',index_col=0) # países como index
df_2 = pd.read_csv('/kaggle/input/world-happiness-report-2021/world-happiness-report.csv',index_col=0) # países como index

# Renomenado as colunas para facilitar o trabalho
df.rename(columns={'Country name': 'Country'}, inplace=True)
df2.rename(columns={'Country name': 'Country'}, inplace=True)
df_1.rename(columns={'Country name': 'Country'}, inplace=True)
df_2.rename(columns={'Country name': 'Country'}, inplace=True)

In [ ]:
# Visão Geral dos Dados de países como índice
df_1.head()

In [ ]:
# Analisando o passado recente (2021 - 2020)
felicidade_2021 = df_1['Ladder score']
# felicidade_2021
felicidade_2020 = df_2.loc[df_2['year'] == 2020]['Life Ladder']
felicidade_2020

In [ ]:
# Criando um dataframe com as diferenças de um ano para outro e retirando os valores que não são números.
diff = pd.DataFrame(felicidade_2021 - felicidade_2020,columns=['diff']).dropna()
diff.describe()

In [ ]:
# Mostrar qual país subiu mais no ranking
print((felicidade_2021['Philippines'] - felicidade_2020['Philippines']).round(2))
print(diff['diff'].idxmax())
print(diff['diff']['Philippines'].round(2))

In [ ]:
# Mostrar qual país desceu mais no ranking
print(diff['diff'].idxmin())
print(diff['diff']['Zambia'].round(2))

In [ ]:
df[['Country','Ladder score','Generosity']].head(10)

In [ ]:
df.iloc[df['Generosity'].idxmax()]

In [ ]:
# Mostrar a correlação entre Felicidade x Generosidade
df.corrwith(df['Generosity'],method='pearson')['Ladder score']

In [ ]:
# Correlação geral
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2).set_properties(**{'font-size': '12pt'})

In [ ]:
# Comparar o Brasil com o Brics
df_1.loc[['Brazil','Russia','India','China','South Africa']].sort_values(['Ladder score'], ascending = False)

In [ ]:
# será considerado o dataset criado anteriormente. Primeiro uma cópia é feita para evitar 
# problemas com o dataset original.

dfm_2807 = df.copy(deep=True)
dfm_2807.head(5)

In [ ]:
# Obtendo os países extremos em felicidade
subset1 = dfm_2807.loc[:,['Country','Ladder score']] # Separo a região do dataframe que quero
subset1 = subset1.sort_values(by='Ladder score', ascending=False) # Ordeno os valores do maior para o menor Ladder Score
print('País mais feliz:')
print(subset1.iloc[0]) # Imprimindo o país mais feliz
print('\nPaís menos feliz:')
print(subset1.iloc[-1]) # Imprimindo o país menos feliz

In [ ]:
# Obtendo como o Brasil está colocado quando comparado com o restante da América do Sul
dfm_2807['Regional indicator'].drop_duplicates() # Obtendo todas as possíveis regiões

In [ ]:
# Como visto, não há uma entrada específica para a América Latina. Logo, devemos filtrar estes países.
subset2 = dfm_2807[dfm_2807['Regional indicator'] == 'Latin America and Caribbean'] # Obtendo os países da América Latina
subset2 = subset2[subset2['Country'].isin(['Brazil','Argentina','Uruguay',\
                                           'Chile','Colombia','Peru',\
                                           'Ecuador','Paraguay','Venezuela',\
                                           'Bolivia'])] # Restringindo aos países da América do Sul
subset2 = subset2.sort_values(by='Ladder score', ascending=False) # Ordenando os dados do maior para o menor Ladder Score
print('Países mais felizes na América do Sul:')
print(subset2.reset_index()[['Country','Ladder score']]) # Mostrando o resultado, ordenado do mais feliz para o menos Feliz

In [ ]:
# Comparando a Felicidade com a Corrupção
subset3 = dfm_2807[['Country','Ladder score','Perceptions of corruption']] # Retirando os dados de interesse
subset3.plot(kind='scatter', x='Ladder score', y='Perceptions of corruption') # Plotando com o Matplotlib

# Sobre o gráfico: é um gráfico de duas coordenadas, que permite traçar curvas analíticas para encontrar uma relação.

In [ ]:
# Para encontrar os pontos fora da curva:
import plotly.express as px

px.scatter(subset3, x='Ladder score', y='Perceptions of corruption', hover_data=['Country']) # PLotando um gráfico interativo